# Loops & Orbits &mdash; Week 3 &mdash; Day 4 &mdash; Jupyter Notebook

## Rocket Burns for Transfer Orbits with `pandas` for Data Visualization

Our last topic is the rocket burns needed to leave a transfer orbit. The idea we are going to implement is that our satellite is in a low circular orbit and we want to compute the rocket burns needed to get to a larger circular orbit. It turns out this can be achieved with two burns, the first burn puts the satellite into an elliptical orbit, and the second burn circularizes that orbit.

This is a kind of orbital insertion. It is far more difficult to do orbital insertion for interplanetary travel.

First, grab all of the imports. **Just execute the imports.** There is nothing here for you to change.

In [ ]:
# Important math functions and the constant pi:
from math import sin, cos, atan2, sqrt, pi

# Use numpy library for dealing with arrays of floats:
import numpy as np

# Today we are adding Pandas for data presentation:
import pandas as pd

# The following import statement makes the plotting library available to us. There is also a mysterious
# statement beginning with % to work around a known Jupyter bug: https://github.com/jupyter/notebook/issues/3670
%matplotlib inline
import matplotlib.pyplot as plt

# Use unit testing framework to test each function we write:
import unittest

Next, define all the constants. **Just execute these assignments.**

All of the constants are now in kilometers instead of meters.

All times are now in minutes.

Conversion factors to standard MKS units are included at the bottom of the cell.

In [ ]:
# The following two parameters are get us the strength of gravity at various heights:
g = 35.32                # 35.32 km/min**2 at Earth surface
radius_of_earth = 6371.0 # Earth radius in km

# The following two variables define the cannonball launch conditions:
initial_position = [0.0, radius_of_earth + 500.0]  # satellite begins at a 500 km high orbit
initial_velocity = [456.7, 0.0]                    # 456.7 km/min maintains the 500 km high orbit

# Properties of burns
burns = [(92, 3.8), (1800.0, 1.0)] # The second burn is ridiculously late. You will be fixing that.
burn_strength = 20.0

# The following two parameters establish the simulation time step and its maximum total duration:
delta_t = 0.05                # simulation time step in minutes --- 0.05 is every three seconds
# For one burn resulting in an elliptical orbit, 5600 is a good value (that is 280 minutes)
simulation_time_steps = 5600  # For two-burn transfer orbit, change this to 9600 for 480 minutes (that is 8 hours)

# rocket mass
rocket_mass = 30000 # Mercury-Redstone launch mass in kg

# Because we are working in kilometers and minutes, if we want energy in Joules, we need to convert
meters_per_kilometer = 1000.0 # needed for conversion to Joules -- two lines below
seconds_per_minute = 60.0 # needed for conversion to Joules -- one line below
conversion_to_joules = (meters_per_kilometer / seconds_per_minute)**2

## Functions for Working with Vectors

All of these have already appeared in previous notebooks. **Just execute the cell.**

In [ ]:
# computes horizontal and vertical components of a vector and returns them as a tuple
def vector_from_length_and_angle(length: float, angle: float) -> np.ndarray:
    # we are working in degrees -- python's are expecting radians -- convert degrees to radian
    angle_in_radians = angle * pi / 180.0
    x_component = length * cos(angle_in_radians)
    y_component = length * sin(angle_in_radians)
    return np.array([x_component, y_component])

# get angle from components using atan2 version of arctangent function -- result is converted to degrees
def angle_from_vector(vector: np.ndarray) -> float:
    # use the arctangent function
    angle_in_radians = atan2(vector[1], vector[0])  
    # we are working in degrees -- python's functions return radians -- convert radians to degrees
    angle = angle_in_radians * 180.0 / pi
    # return the result
    return angle

# get length from components using Pythagorean theorem
def length_from_vector(vector: np.ndarray) -> float:
    length_squared = np.sum(vector**2)
    return sqrt(length_squared)

### Unit tests ###

class VectorTests(unittest.TestCase):
    
    def test_vector_from_length_and_angle(self):
        vector = vector_from_length_and_angle(1000.0, 30.0)
        expected_vector = np.array([866.02540378, 500.0])
        self.assertTrue(np.allclose(vector, expected_vector),
                        msg="Expected {}, but got {}".format(expected_vector, vector))
        
    def test_angle_from_vector(self):
        angle = angle_from_vector(np.array([866.02540378, 500.0]))
        self.assertAlmostEqual(angle, 30.0)
        
    def test_length_from_vector(self):
        length = length_from_vector(np.array([5.0, 12.0]))
        self.assertAlmostEqual(length, 13.0)

testSuite = unittest.TestLoader().loadTestsFromName("__main__.VectorTests")
testRunner = unittest.TextTestRunner(verbosity=2)
testRunner.run(testSuite)

## Functions to Get Acceleration due to Gravity

These are functions you implemented in the previous notebook. **Just execute the cell.**

In [ ]:
# The fancy function declaration with the : after position and the -> before float
# says what the type of the argument and the return value is. The older, simpler way we were
# declaring functions would have just begun with:

# def strength_of_gravity(position):
      # this function encodes the strength of gravity....
    
# The new way of doing declarations is longer, and it is a matter of taste whether it helps readability.
# The new way of doing declarations says the position arguments is an np.ndarray (a numpy
# n-dimensional array). It says that the function returns a Boolean.
def strength_of_gravity(position: np.ndarray) -> float:
    # this function encodes the strength of gravity as a function of distance from the center of the Earth
    radius = length_from_vector(position)
    strength = g * radius_of_earth**2 / radius**2
    return strength

def direction_of_gravity(position: np.ndarray) -> np.ndarray:
    # this function encodes the direction of gravity (the angle)
    # gravity is attractive -- it always points toward the center of the Earth
    direction = angle_from_vector(position) + 180.0
    return direction

def acceleration_from_gravity(position: np.ndarray) -> np.ndarray:
    # using the strength and direction functions you have just implemented compute and
    # returns a 2x1 array for the acceleration of gravity
    strength = strength_of_gravity(position)
    direction = direction_of_gravity(position)
    acceleration = vector_from_length_and_angle(strength, direction)
    return acceleration

### Unit tests ###

class GravityTests(unittest.TestCase):

    # tests of strength

    def test_strength_of_gravity(self):
        position = np.array([radius_of_earth, 0.0])
        strength = strength_of_gravity(position)
        self.assertAlmostEqual(strength, g)

    def test_strength_of_gravity_high_up(self):
        position = np.array([0.0, 2.0 * radius_of_earth])
        strength = strength_of_gravity(position)
        self.assertAlmostEqual(strength, 0.25 * g)
        
    # tests of direction
        
    def test_direction_of_gravity_left(self):
        position = np.array([100.0, 0.0])
        direction = direction_of_gravity(position)
        self.assertEqual(direction, 180.0)

    def test_direction_of_gravity_up(self):
        position = np.array([0.0, -100.0])
        direction = direction_of_gravity(position)
        self.assertEqual(direction, 90.0)
        
    # tests of acceleration

    def test_acceleration_from_gravity_g_left(self):
        position = np.array([radius_of_earth, 0.0])
        acceleration = acceleration_from_gravity(position)
        expected_acceleration = np.array([-g, 0.0])
        self.assertTrue(np.allclose(acceleration, expected_acceleration),
                        msg="Expected {}, but got {}".format(expected_acceleration, acceleration))

    def test_acceleration_from_gravity_4g_up(self):
        position = np.array([0.0, -radius_of_earth / 2.0])
        acceleration = acceleration_from_gravity(position)
        expected_acceleration = np.array([0.0, 4.0 * g])
        self.assertTrue(np.allclose(acceleration, expected_acceleration),
                        msg="Expected {}, but got {}".format(expected_acceleration, acceleration))

testSuite = unittest.TestLoader().loadTestsFromName("__main__.GravityTests")
testRunner = unittest.TextTestRunner(verbosity=2)
testRunner.run(testSuite)


## Functions to get Acceleration due to Thrust

**You will implement the first of the three functions below.** There are six unit tests to make sure that you have done the burn times correctly.

In [ ]:
def strength_of_thrust(time: float) -> float:
    # This function requires some serious thought. I have gotten you started with a single line that is correct.
    # Work on finishing the function. The unit tests will pass when the function is right.
    for burn in burns:
        # burn is a tuple, unpack it.
        return 0.0
    # time was not during a burn
    return 0.0

def direction_of_thrust(velocity: np.ndarray) -> np.ndarray:
    # this function encodes the direction of thrust (the angle)
    direction = angle_from_vector(velocity)
    return direction

def acceleration_from_thrust(time: float, velocity: np.ndarray) -> np.ndarray:
    # using the strength and direction function, compute
    # a 2x1 array for the acceleration from thrust
    strength = strength_of_thrust(time)
    direction = direction_of_thrust(velocity)
    acceleration = vector_from_length_and_angle(strength, direction)
    return acceleration


### Unit tests ###

class BurnTests(unittest.TestCase):

    # tests of strength

    def test_strength_of_thrust_before_first_burn(self):
        first_burn = burns[0]
        burn_begin_time, burn_duration = first_burn
        time = burn_begin_time - 0.1
        strength = strength_of_thrust(time)
        self.assertAlmostEqual(strength, 0.0)

    def test_strength_of_thrust_during_first_burn(self):
        first_burn = burns[0]
        burn_begin_time, burn_duration = first_burn
        time = burn_begin_time + 0.5 * burn_duration
        strength = strength_of_thrust(time)
        self.assertAlmostEqual(strength, burn_strength)
        
    def test_strength_of_thrust_after_first_burn(self):
        first_burn = burns[0]
        burn_begin_time, burn_duration = first_burn
        time = burn_begin_time + burn_duration + 0.1
        strength = strength_of_thrust(time)
        self.assertAlmostEqual(strength, 0.0)

    def test_strength_of_thrust_before_last_burn(self):
        last_burn = burns[-1]
        burn_begin_time, burn_duration = last_burn
        time = burn_begin_time - 0.1
        strength = strength_of_thrust(time)
        self.assertAlmostEqual(strength, 0.0)

    def test_strength_of_thrust_during_last_burn(self):
        last_burn = burns[-1]
        burn_begin_time, burn_duration = last_burn
        time = burn_begin_time + 0.5 * burn_duration
        strength = strength_of_thrust(time)
        self.assertAlmostEqual(strength, burn_strength)
        
    def test_strength_of_thrust_after_last_burn(self):
        last_burn = burns[-1]
        burn_begin_time, burn_duration = last_burn
        time = burn_begin_time + burn_duration + 0.1
        strength = strength_of_thrust(time)
        self.assertAlmostEqual(strength, 0.0)

testSuite = unittest.TestLoader().loadTestsFromName("__main__.BurnTests")
testRunner = unittest.TextTestRunner(verbosity=2)
testRunner.run(testSuite)


## The While Loop That Does the Work &mdash; 2nd Order Runge-Kutta

There is nothing for you to change below. **Just execute the cell.** You implemented 2nd Order Runge-Kutta last time.

In [ ]:
# Initialize the x and y velocities
velocities = np.zeros((simulation_time_steps, 2))
velocities[0] = initial_velocity

# Initialize the x and y positions
positions = np.zeros((simulation_time_steps, 2))
positions[0] = initial_position

# Initialize the times
times = np.zeros((simulation_time_steps,))

# The 0th elements of velocities, positions, and times are already filled in, so start with 1
for i in range(1, simulation_time_steps):
    #
    # get all the before values
    #
    # velocities
    before_velocity = velocities[i - 1]
    # positions
    before_position = positions[i - 1]
    # time
    before_time = times[i - 1]
    #
    # fundamental change for 2nd-order Runge-Kutta -- estimate mid_position
    #
    mid_position = before_position + 0.5 * delta_t * before_velocity
    #
    # need before_acceleration
    #
    before_acceleration = acceleration_from_gravity(before_position) + acceleration_from_thrust(before_time, before_velocity)
    #
    # now can get mid_velocity estimate
    #
    mid_velocity = before_velocity + 0.5 * delta_t * before_acceleration
    #
    # also compute mid_time
    #
    mid_time = before_time + 0.5 * delta_t
    #
    # Using mid_position, mid_velocity, and mid_time compute mid_acceleration
    #
    mid_acceleration = acceleration_from_gravity(mid_position) + acceleration_from_thrust(mid_time, mid_velocity)
    #
    # calculate after_velocity using mid_acceleration
    #
    after_velocity = before_velocity + delta_t * mid_acceleration
    #
    # calculate after_position using the democratic combination 
    # of (after_velocity + before_velocity) / 2
    #
    after_position = before_position + delta_t * (after_velocity + before_velocity) / 2.0
    #
    # update time
    #
    after_time = before_time + delta_t
    #
    # assign the after values into their lists
    #
    velocities[i] = after_velocity
    positions[i] = after_position
    times[i] = after_time


## Construct a Panda DataFrame

In [ ]:
df = pd.DataFrame({
    'time' : times[:],
    'x_pos' : positions[:, 0],
    'y_pos' : positions[:, 1],
    'x_vel' : velocities[:, 0],
    'y_vel' : velocities[:, 1]
})

# radius
df['radius'] = (df.x_pos**2 + df.y_pos**2)**0.5
# use the radius to get the elevation
df['elevation'] = df.radius - radius_of_earth
# get the speed
df['speed'] = (df.x_vel**2 + df.y_vel**2)**0.5
# construct a unit vector
df['xs_pos'] = df.x_pos / df.radius
df['ys_pos'] = df.y_pos / df.radius
# dot the unit vector into the velocity to get the radial speed (rate of altitude change)
df['radial_speed'] = df.x_vel * df.xs_pos + df.y_vel * df.ys_pos



## Visualize the Transfer Orbit

This is the same code (pretty much) as was in the previous notebook. Instead of just ellipses, we now have a circular orbit, and an elliptical orbit. The first rocket burn transitions from the low circular orbit to the elliptical orbit.

You will have to play with the notebook to do the following directions.

### Directions

**At the peak of the elliptical orbit, you need to figure out the second rocket burn that allows that orbit to be stabilized as a new higher circular orbit.**

* Step 1: Increase simulation_time_steps to 9600
* Step 2: Study the elevation plot a ways further down in this notebook. At what time does the rocket achieve its maximum elevation? Change the second burn to start a minute or two before maximum elevation is achieved.
* Step 3: Increase the second burn duration just enough that the new orbit is circular.

In [ ]:
sampled_df = df[ df.index % 100 == 0 ]

plt.figure(figsize=(12, 12))

plt.scatter(sampled_df.x_pos, sampled_df.y_pos)

plt.xlabel("x position (km)")
plt.ylabel("y position (km)")

# Some gibberish that draws a big blue circle representing the Earth:
earth = plt.Circle((0, 0), radius_of_earth, color='b')
plt.gcf().gca().add_artist(earth)

# Make the plot big enough to show the original and final orbits:
plot_limit = 8000
plt.xlim(-2.2 * plot_limit, 2.2 * plot_limit)
plt.ylim(-2.2 * plot_limit, 2.2 * plot_limit)

plt.show()

## Making Plots with Pandas

The following is a nice example of making three subplots that share a common horizontal axis using pandas and matplotlib. As one of your last tasks in this notebook you will be making three more plots to study kinetic energy, potential energy, and total energy.

In [ ]:
# Satellite Elevation and Speed

fig, axs = plt.subplots(3, figsize=(12, 12), sharex=True)
fig.suptitle('Satellite Elevation and Speed')

# first subplot
axs[0].set_ylabel('elevation (km)')
axs[0].plot(df.time, df.elevation)

# second subplot
axs[1].set_ylabel('speed (km/min)')
axs[1].plot(df.time, df.speed)

# third subplot
axs[2].set_ylabel('radial speed (km/min)')
axs[2].plot(df.time, df.radial_speed)

# Only set x label on the third subplot
axs[2].set_xlabel('time (minutes)')

"Roger, Houston"

## Energy Calculations

### From Speed Construct Kinetic Energy

The formula for kinetic energy is on the board.

Make `ke` a new column in the Panda data frame.

This is a one-liner in Pandas. Just look at how it was done in the section titled "Construct a Panda Data Frame."

In [ ]:
# Make 'ke'

### From Radius Construct Potential Energy

The formula for potential energy is on the board.

Make `pe` a new column in the Panda data frame. This is another one-liner.

In [ ]:
# Make 'pe'

### From Potential Energy and Kinetic Energy get Total Energy

Make `te` a new column in the Panda data frame. It's just the sum of the previous two.

In [ ]:
# Make 'te'

## Make a New Set of Three Plots

Copy-and-paste the code from the section titled "Making Plots with Pandas."

Adjust it to show kinetic energy, potential energy, and total energy.

In [ ]:
# Satellite Kinetic and Potential Energy

## Checkout Questions

* Study the total energy graph. When was total energy not conserved?
* Energy is always conserved. Where did this energy come from?

## Conclusion

We have gotten to where we planned in the [syllabus](http://physics.stmarys-ca.edu/faculty/brianhill/courses/Jan033/20J/index.html). I hope you have enjoyed the richness that is possible using computational methods.

The instruction in this course has moved extremely fast, both because I always try to give students their money's worth and because Jan Term moves at 4 times the pace of a normal semester. It's ok if it hasn't all gelled.

By continued study of all of the notebooks we have done &mdash; all of which are posted on the [loops and orbits repo](https://github.com/observatree/loops-and-orbits) &mdash; I'm confident you can cement in the material.

Once you get it cemented in, you will know as much as the typical science professor about making Python and Jupyter do your bidding. Saint Mary's faculty who specialize in computational methods, like Prof. Aaron Lee, Prof. Jessica Coyle, and Prof. Andr&aacute;s Margitay-Becht to name a few, can help you learn much more. Ask around in your department. Show the ones that are interested what you have learned. Doors will open and new ideas will flow.

Finally, I very much look forward to helping you apply the methods you have seen in these three weeks to the projects you choose in the final week of this course. My door is open most of the time, and you can make appointments for help.